# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [2]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [4]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [5]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/la-decision-des-decisions-de-lebron-james-un-coup-de-publicite-trompeur-11612545',
  'La "décision des décisions" de LeBron James\xa0? Un coup de publicité trompeur'],
 ['https://www.rtbf.be/article/derriere-les-discours-rassurants-l-europe-inquiete-de-la-situation-politique-en-france-11612383',
  'Derrière les discours rassurants, l’Europe inquiète de la situation politique en France'],
 ['https://www.rtbf.be/article/novak-djokovic-rejoint-zizou-bergs-en-quarts-de-finale-a-shanghai-11612565',
  'Novak Djokovic rejoint Zizou Bergs en quarts de finale à Shanghai'],
 ['https://www.rtbf.be/article/avec-icecube-le-physicien-belge-francis-halzen-etait-a-une-particule-fantome-du-prix-nobel-de-physique-11612432',
  'Avec IceCube, le physicien belge Francis Halzen était à une «\xa0particule fantôme\xa0» du Prix Nobel de physique'],
 ['https://www.rtbf.be/article/tadej-pogacar-vainqueur-des-trois-vallees-varesines-grace-a-une-attaque-en-descente-11612473',
  'Tade

## Création d'un dataframe avec les liens et les titres des articles


In [6]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/la-decision-des-de...,"La ""décision des décisions"" de LeBron James ? ..."
1,https://www.rtbf.be/article/derriere-les-disco...,"Derrière les discours rassurants, l’Europe inq..."
2,https://www.rtbf.be/article/novak-djokovic-rej...,Novak Djokovic rejoint Zizou Bergs en quarts d...
3,https://www.rtbf.be/article/avec-icecube-le-ph...,"Avec IceCube, le physicien belge Francis Halze..."
4,https://www.rtbf.be/article/tadej-pogacar-vain...,Tadej Pogacar vainqueur des Trois Vallées Varé...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,"Nos chiens, ces héros"
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Jardins & Loisirs
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Le 13h
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Plus belle la vie S13


In [7]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [8]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

La "décision des décisions" de LeBron James ? Un coup de publicité trompeur
Dans cette fameuse vidéo mise en ligne lundi, le " King ", âgé de 40 ans, entrait dans le champ et s’asseyait, silencieux, face à un interlocuteur avant qu’apparaisse à l’écran une incrustation " La deuxième décision, prochainement ".
Ce titre faisait le parallèle avec " The Decision " (" La Décision "), célèbre explication de son départ en 2010 de Cleveland pour Miami, où il a conquis deux de ses quatre titres NBA.
D’annonce choc, il s’agissait en fait d’un partenariat entre la star du basket et une célèbre marque de cognac mis en scène dans une nouvelle vidéo de dix secondes.
Entre ces deux vidéos, le monde du basket s’est interrogé sur ce que " LBJ " allait annoncer tandis que s’envolaient les prix des billets des Los Angeles Lakers, la franchise où il évolue depuis 2018 et avec laquelle il se prépare à une 23e saison en NBA.
Selon le site de vente Tickpick, les tarifs pour le dernier match à domicile des La

### Nettoyage du texte à l'aide d'expressions régulières

In [9]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Dans cette fameuse vidéo mise en ligne lundi, le " King ", âgé de 40 ans, entrait dans le champ et s’asseyait, silencieux, face à un interlocuteur avant qu’apparaisse à l’écran une incrustation " La deuxième décision, prochainement ".
Ce titre faisait le parallèle avec " The Decision " (" La Décision "), célèbre explication de son départ en 2010 de Cleveland pour Miami, où il a conquis deux de ses quatre titres NBA.
D’annonce choc, il s’agissait en fait d’un partenariat entre la star du basket et une célèbre marque de cognac mis en scène dans une nouvelle vidéo de dix secondes.
Entre ces deux vidéos, le monde du basket s’est interrogé sur ce que " LBJ " allait annoncer tandis que s’envolaient les prix des billets des Los Angeles Lakers, la franchise où il évolue depuis 2018 et avec laquelle il se prépare à une 23e saison en NBA.
Selon le site de vente Tickpick, les tarifs pour le dernier match à domicile des Lakers lors de la saison régulière, le 12 avril prochain contre l’Utah Jazz, o

### Création d'un fichier avec le contenu de l'article


In [10]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp